## Google Analytics Customer Revenue Prediction

## 1. Getting the data

In [25]:
#### Loading the Required Packages ############
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys

from pandas.io.json import json_normalize
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

#### Function to convert XML to dataframes 

In [21]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [str(column)+"."+str(subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print("Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

#### Calling the input train and test xml files to convert to dataframe

In [18]:
#train = load_df(os.getcwd()+'\\train.csv')
#test = load_df(os.getcwd()+'/test.csv')

#### Saving a CSV copy of generated dataframes

In [25]:
#train.to_csv('train_extracted.csv',encoding='utf8')
#test.to_csv('test_extracted.csv', encoding = 'utf8')

## Reading saved the data
train = pd.read_csv('train_extracted.csv',encoding='utf8',dtype={'fullVisitorId': 'str'})
test = pd.read_csv('test_extracted.csv', encoding = 'utf8',dtype={'fullVisitorId': 'str'})

#### Combining train and test data

For ease in computation of data transformation and other changes, we combine the train and test

In [27]:
train['data_source'] = 'train'
test['data_source'] = 'test'

all_data = pd.concat([train,test], axis = 0)

In [28]:
all_data.shape

(1708337, 56)

## 2. Analyzing the Data

#### Emtpy values 

Checking the ratio of NULL/empty values

In [19]:
100 * float(train['totals_transactionRevenue'].isnull().sum())/len(train['totals_transactionRevenue'])

NameError: name 'train' is not defined

#### Data Snapshot


In [37]:
#all_data.drop([u'Unnamed: 0'], axis = 1, inplace = True)
all_data.head()

,channelGrouping,data_source,date,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,...,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,visitId,visitNumber,visitStartTime
0,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472830385,1,1472830385
1,Organic Search,train,20160902,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472880147,1,1472880147
2,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,(not provided),organic,NaN,google,1472865386,1,1472865386
3,Organic Search,train,20160902,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,...,(not set),NaN,NaN,google + online,organic,NaN,google,1472881213,1,1472881213
4,Organic Search,train,20160902,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,...,(not set),NaN,True,(not provided),organic,NaN,google,1472822600,2,1472822600


In [20]:
#### Distribution of the 'visitNumber' variable
all_data.visitNumber.describe()

NameError: name 'all_data' is not defined

#### Response variable

In [56]:
all_data['totals_transactionRevenue'] = pd.to_numeric(all_data['totals_transactionRevenue'])
all_data['totals_transactionRevenue'].describe()

count    1.151500e+04
mean     1.337448e+08
std      4.482852e+08
min      1.000000e+04
25%      2.493000e+07
50%      4.945000e+07
75%      1.076550e+08
max      2.312950e+10
Name: totals_transactionRevenue, dtype: float64

#### Distribution of other columns apart from Response variable

In [21]:
for each in all_data_upd.columns:
    print(each)
    print(all_data_upd[each].value_counts())
    print("\n")

NameError: name 'all_data_upd' is not defined

## 3. Data Pre-processing

#### Finding columns with just one value in it

Finding the columns with  <br>
i) If the number of unique values is just one <br>
ii) More than 50% of NA values <br><br>

Dropping such columns



In [55]:
### Selecting columns with just one unique value and those with NULL values greater than 50% of the dataset
drop_columns = [each for each in all_data.columns if len(train[each].unique()) <= 2 or all_data[each].isnull().sum() > 0.5 * all_data.shape[0]]
drop_columns = [each for each in drop_columns if each not in ['totals_transactionRevenue','data_source']]

In [57]:
all_data_upd = all_data.drop(drop_columns, axis = 1)

#### Feature Engineering: Creating date variables

In [59]:
all_data_upd['year'] = all_data_upd.date.apply(lambda x: int(str(x)[0:4]))
all_data_upd['month'] = all_data_upd.date.apply(lambda x: int(str(x)[4:6]))
all_data_upd['day'] = all_data_upd.date.apply(lambda x: int(str(x)[6:8]))

#### Cleaning and imputing train and test separately - since we do not want to use the test data to impute the training data

In [60]:
train_upd = all_data_upd[all_data_upd.data_source == 'train']
test_upd = all_data_upd[all_data_upd.data_source == 'test']

print(train_upd.shape)
print(test_upd.shape)

(903653, 27)
(804684, 27)


Viewing Training dataset

In [61]:
train_upd.head()

,channelGrouping,data_source,date,device_browser,device_deviceCategory,device_operatingSystem,fullVisitorId,geoNetwork_city,geoNetwork_continent,geoNetwork_country,...,totals_transactionRevenue,trafficSource_campaign,trafficSource_medium,trafficSource_source,visitId,visitNumber,visitStartTime,year,month,day
0,Organic Search,train,20160902,Chrome,desktop,Windows,1131660440785968503,Izmir,Asia,Turkey,...,NaN,(not set),organic,google,1472830385,1,1472830385,2016,9,2
1,Organic Search,train,20160902,Firefox,desktop,Macintosh,377306020877927890,not available in demo dataset,Oceania,Australia,...,NaN,(not set),organic,google,1472880147,1,1472880147,2016,9,2
2,Organic Search,train,20160902,Chrome,desktop,Windows,3895546263509774583,Madrid,Europe,Spain,...,NaN,(not set),organic,google,1472865386,1,1472865386,2016,9,2
3,Organic Search,train,20160902,UC Browser,desktop,Linux,4763447161404445595,not available in demo dataset,Asia,Indonesia,...,NaN,(not set),organic,google,1472881213,1,1472881213,2016,9,2
4,Organic Search,train,20160902,Chrome,mobile,Android,27294437909732085,not available in demo dataset,Europe,United Kingdom,...,NaN,(not set),organic,google,1472822600,2,1472822600,2016,9,2


#### Imputation for Trainining dataset

In [62]:
## Cleaning the character columns in the data

#Float columns
float_cols = []


for each in train_upd.columns:
    
    if(each == 'data_source'):
        continue
        
    if(train_upd[each].dtype == 'O' and len(train_upd[each].unique()) < 500000):
        print(each)
        ### Skipping the values are actually numeric or float
        
        if( not (len(pd.to_numeric(train_upd[each],errors = 'coerce').unique()) < 0.5 * len(train_upd[each].unique())) ):
            
            train_upd.loc[:,each] = train_upd[each].fillna(train_upd[each].mean())
            float_cols.append(each)
            
        else:


            print(len(train_upd[each].unique()))
            train_upd.loc[:,each] = train_upd[each].apply(lambda x: x.lower().strip())
            extreme_values = train_upd[each].value_counts()[train_upd[each].value_counts() < 10].index

            ## Replacing the outliers (extreme) values with 'Others'

            train_upd.loc[:,each] = train_upd[each].apply(lambda x: np.where(x in extreme_values, 'Others',x))

channelGrouping
8


C:\Users\arvra\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


device_browser
54
device_deviceCategory
3
device_operatingSystem
20
geoNetwork_city
649
geoNetwork_continent
6
geoNetwork_country
222
geoNetwork_metro
94
geoNetwork_networkDomain
28064
geoNetwork_region
376
geoNetwork_subContinent
23
trafficSource_campaign
10
trafficSource_medium
7
trafficSource_source
380


#### Imputation for Test dataset

In [63]:
## Cleaning the character columns in the data

for each in test_upd.columns:
    
    
    if(each == 'data_source'):
        continue
    
    print(each)
    if(test_upd[each].dtype == 'O' and len(test_upd[each].unique()) < 500000):
            
        ### Skipping the values are actually numeric or float
        
        ### Identifying the columns as numeric/float as per the training data
        if(each in float_cols):
            
            train_upd[each] = train_upd[each].fillna(train_upd[each].mean())
        else:


            print(len(train_upd[each].unique()))
            train_upd[each] = train_upd[each].apply(lambda x: x.lower().strip())
            extreme_values = train_upd[each].value_counts()[train_upd[each].value_counts() < 10].index

            ## Replacing the outliers (extreme) values with 'Others'

            train_upd[each] = train_upd[each].apply(lambda x: np.where(x in extreme_values, 'Others',x))

channelGrouping
8


C:\Users\arvra\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\arvra\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


date
device_browser
27
device_deviceCategory
3
device_operatingSystem
15
fullVisitorId
geoNetwork_city
482
geoNetwork_continent
6
geoNetwork_country
188
geoNetwork_metro
79
geoNetwork_networkDomain
3017
geoNetwork_region
318
geoNetwork_subContinent
23
sessionId
totals_hits
totals_pageviews
totals_transactionRevenue
trafficSource_campaign
9
trafficSource_medium
7
trafficSource_source
124
visitId
visitNumber
visitStartTime
year
month
day


#### Combining the data again

In [64]:
all_data_upd = pd.concat([train_upd,test_upd], axis = 0)

In [22]:
## Removing all the columns that has more than ~200 distinct objects
#(since those will create multiple columns while creating one hot vectors)

## Dropping such columns
all_data_upd = all_data_upd.drop(['device_browser','geoNetwork_metro','geoNetwork_networkDomain','geoNetwork_region',
                                  'trafficSource_source','geoNetwork_city','geoNetwork_country'], axis = 1)

## Getting the datashape
all_data_upd.shape

NameError: name 'all_data_upd' is not defined

In [68]:
all_data_upd['totals_transactionRevenue'] = all_data_upd['totals_transactionRevenue'].fillna(0)
all_data_upd = all_data_upd.fillna(0)

In [69]:
### Removing id columns
id_cols = ['sessionId','fullVisitorId','visitId','date']
all_data_dummies = pd.get_dummies(all_data_upd.drop(id_cols, axis = 1), dummy_na = True)

In [70]:
all_data_dummies.shape

(1708337, 175)

#### Saving the transformed train data

In [2]:
#all_data_dummies.to_csv('all_data_dummies.csv', index = False, encoding = 'utf8')
all_data_dummies = pd.read_csv("all_data_dummies.csv", encoding = 'utf8')

In [3]:
all_data_dummies.shape

(1708337, 175)

#### Dividing the data into train and test

In [5]:
train_dummies = all_data_dummies[all_data_dummies.data_source_train == 1]
test_dummies = all_data_dummies[all_data_dummies.data_source_test == 1]

# train_dummies = all_data_upd[all_data_upd.data_source == 'train']
# test_dummies = all_data_upd[all_data_upd.data_source == 'test']

### Creating the dataset that is model-ready

In [ ]:
train_dummies = train_dummies.drop(['data_source_train','data_source_test','data_source_nan'], axis = 1)
test_dummies = test_dummies.drop(['data_source_train','data_source_test','data_source_nan','totals_transactionRevenue'], axis = 1)

#train_dummies = train_dummies.drop(['data_source'], axis = 1)
#test_dummies = test_dummies.drop(['data_source','totals.transactionRevenue'], axis = 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_dummies.drop(['totals_transactionRevenue'], axis = 1), \
                  train_dummies['totals_transactionRevenue'],test_size = 0.2, random_state = 9)

In [8]:
X_train.head()

,totals_hits,totals_pageviews,visitNumber,visitStartTime,year,month,day,channelGrouping_(Other),channelGrouping_(other),channelGrouping_Affiliates,...,trafficSource_campaign_test-liyuhz,trafficSource_campaign_nan,trafficSource_medium_(none),trafficSource_medium_(not set),trafficSource_medium_affiliate,trafficSource_medium_cpc,trafficSource_medium_cpm,trafficSource_medium_organic,trafficSource_medium_referral,trafficSource_medium_nan
40485,4,4.0,1,1498884588,2017,6,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
784765,1,1.0,1,1501200752,2017,7,27,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103499,1,1.0,1,1499169217,2017,7,4,0,0,0,...,0,0,0,0,0,0,0,1,0,0
363033,1,1.0,4,1482550584,2016,12,23,0,0,0,...,0,0,1,0,0,0,0,0,0,0
886064,3,2.0,5,1500310627,2017,7,17,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
print(X_train.shape)
print(y_train.shape)

(722922, 171)
(722922,)


In [26]:
#### Response variable for logistic regression
y_train_logistic = np.where(y_train != 0,1,0)

## 4. Model Building

There will be two types of models<br><br>
1) Logistic Regression to predict where we have to predict if the value is zero or non-zero <br>
2) Linear Regression to predict the actual Revenue <br>
    &emsp; 2.1) OLS <br>
    &emsp; 2.2) Random forest <br>
    &emsp; 2.3) Splines <br>

### 4.1 Logistic Regression

 Model to find where we have 0 revenue and 1 (non-zero) revenue

In [ ]:
log_model = LogisticRegression(C = 0.01, solver = 'newton-cg')
log_model.fit(X_train,y_train_logistic)

#### Predicting the values - Obtaining prediction probability to manually specify a threshold value

np.sum(log_model.predict_proba(X_train))

C:\Users\arvra\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:461: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\arvra\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


### 4.2 Regression Models
#### 4.2.1 OLS Models

In [23]:
lr_model = LinearRegression(n_jobs = -1)

In [27]:
lr_model.fit(X_train,y_train)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [12]:
predict = lr_model.predict(X_train)
predict_test = lr_model.predict(X_test)

In [13]:
np.histogram(predict)

(array([716501,   5739,    539,     79,     37,     13,      1,      8,
             2,      3], dtype=int64),
 array([-1.25905828e+07,  4.04889368e+07,  9.35684564e+07,  1.46647976e+08,
         1.99727496e+08,  2.52807015e+08,  3.05886535e+08,  3.58966054e+08,
         4.12045574e+08,  4.65125094e+08,  5.18204613e+08]))

In [14]:
lmse = np.sum(np.log((y_train.values - np.where(predict < 1.04889368e+09, 0, predict)) ** 2 + 1))/len(y_train.values)
lmse

0.45331784935534464

In [15]:
lmse = np.sum(np.log((y_test.values - np.where(predict_test < 1.04889368e+9, 0, predict_test)) ** 2 + 1))/len(y_test.values)
lmse

0.4579103000800032

#### Predicting the test class

In [71]:
predict_actual_test = lr_model.predict(test_dummies)
predict_actual_t = np.log(np.where(predict_actual_test < 1e+9, 0,predict_actual_test)+1)

In [72]:
submission = pd.concat([test.fullVisitorId,pd.DataFrame(predict_actual_t)], axis = 1)
submission.columns.values[1]= 'PredictedLogRevenue'

In [73]:
submission_out = submission.groupby('fullVisitorId',as_index = False).sum()

In [74]:
submission_out.to_csv("kaggle_submission.csv", index = False)

#### Random forest

In [75]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf_classifier = RandomForestClassifier()

In [ ]:
#### To do:

## For visualiation: https://www.kaggle.com/pavansanagapati/simple-exploration-lgbm-model-lb-1-4187 
## Implement the globe

## For categorical features - implement the Encoders and proper imputation
## For columns with webties - extract the major website name (google.com/google.in have the same major content google)
## Numerical features - better imputation and try to create more variables


## Futher, implement a logistic regression and then perfrom a non-linear model on top
